### 載入套件

In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
from tensorflow.keras.datasets import fashion_mnist

In [3]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [4]:
labels = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [5]:
x_train.shape

(60000, 28, 28)

### 資料整理
##### 一個記分板為一channel，所以將資料從(28,28)-->(28,28,1)

In [6]:
x_train = x_train.reshape(60000, 28, 28, 1) / 255

In [7]:
x_test = x_test.reshape(10000, 28, 28, 1) / 255

In [8]:
x_train[20].shape

(28, 28, 1)

In [9]:
y_train[20]

3

### 打造神經網路

In [10]:
from tensorflow.keras.utils import to_categorical

In [11]:
#10種資料，0-9
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [12]:
y_train[20]

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.], dtype=float32)

### 讀入必要的函式

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, AveragePooling2D
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD

### Step 1 打造函數學習機 (CNN)¶

In [19]:
model = Sequential()

#### padding='same': 輸入跟輸出都是28*28，filter是3x3輸出32個channel(記分板)且28x28的矩陣>>事實上是(28,28,32)

In [20]:
model.add(Conv2D(32, (3,3), padding='same',
                input_shape=(28,28,1),
                activation='relu'))

#### 將每張記分板切成2*2>>(14,14,32)，整個圖片的尺寸將會縮小2分之1

In [21]:
model.add(AveragePooling2D(pool_size=(2,2)))

#### filter是3x3輸出64個channel(記分板)且14x14的矩陣>>output(14,14,64)

In [22]:
model.add(Conv2D(64, (3,3), padding='same',
                activation='relu'))

#### 每張記分板切成2*2>>(7, 7, 64)

In [23]:
model.add(AveragePooling2D(pool_size=(2,2)))

#### fliter是3x3輸出128個channel(記分板)且7x7的矩陣>>(7,7,128)

In [24]:
model.add(Conv2D(128, (3,3), padding='same',
                activation='relu'))

In [25]:
model.add(AveragePooling2D(pool_size=(2,2)))

In [26]:
#拉平，轉為一維陣列
model.add(Flatten())

In [27]:
# 第一層200個神經元
model.add(Dense(200, activation='relu'))

In [28]:
# 第二層100個神經元
model.add(Dense(100,activation='relu'))

In [29]:
# 第三層50個神經元
model.add(Dense(50,activation='relu'))

In [30]:
# 第四層，輸出層，輸出10個結果，softmax要求輸出的10個數字相加為1
model.add(Dense(10, activation='softmax'))

### 看一下我們的神經網路¶

In [31]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
average_pooling2d (AveragePo (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
average_pooling2d_1 (Average (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 7, 7, 128)         73856     
_________________________________________________________________
average_pooling2d_2 (Average (None, 3, 3, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 1152)              0         
__________

In [32]:
# conv2d_2 (Conv2D):[3*3 (權重) + 1 (bias)]*32
(3*3+1)*32

320

In [33]:
# conv2d_3 (Conv2D):[3*3*32(權重) + 1 (bias)]*64
(3*3*32+1)*64

18496

In [34]:
# conv2d_4 (Conv2D):[3*3*64(權重) + 1 (bias)]*128
(3*3*64+1)*128

73856

In [35]:
model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.08),
             metrics=['accuracy'])

### Step 2. fit

In [36]:
model.fit(x_train, y_train, batch_size=100, epochs=20)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
60000/60000 [==============================] - 122s 2ms/sample - loss: 1.0525 - acc: 0.6133
Epoch 2/20
60000/60000 [==============================] - 61s 1ms/sample - loss: 0.6082 - acc: 0.7671
Epoch 3/20
60000/60000 [==============================] - 61s 1ms/sample - loss: 0.5116 - acc: 0.8063
Epoch 4/20
60000/60000 [==============================] - 61s 1ms/sample - loss: 0.4517 - acc: 0.8305
Epoch 5/20
60000/60000 [==============================] - 61s 1ms/sample - loss: 0.4126 - acc: 0.8428
Epoch 6/20
60000/60000 [==============================] - 61s 1ms/sample - loss: 0.3829 - acc: 0.8560
Epoch 7/20
60000/60000 [==============================] - 61s 1ms/sample - loss: 0.3596 - acc: 0.8656
Epoch 8/20
60000/60000 [==============================] - 62s 1ms/sample - loss: 0.3424 - acc: 0.8716
Epoch 9/20
60000/60000 [==============================] - 61s 1ms/sample - loss: 0.3271 - acc: 0.8762
Epoch 10/20
60000/60000 [========

### Step 3. 預測

In [37]:
result = model.predict_classes(x_test)

In [41]:
def my_predict(n):
    print('CNN 預測是', labels[result[n]])
    X = x_test[n].reshape(28,28)
    plt.imshow(X, cmap='Greys')

In [42]:
from ipywidgets import interact_manual

In [43]:
interact_manual(my_predict, n=(0, 9999))

interactive(children=(IntSlider(value=4999, description='n', max=9999), Button(description='Run Interact', sty…

<function __main__.my_predict(n)>

In [44]:
score = model.evaluate(x_test, y_test)

10000/10000 [==============================] - 6s 566us/sample - loss: 0.2881 - acc: 0.8943


In [45]:
loss, acc = score

In [46]:
print('測試資料的正確率為', acc)

測試資料的正確率為 0.8943


### 透過更改以下設定，model正確率變為0.8943
* 將三層channel數量改為32,64,128
* 使用AveragePooling2D代替MaxPooling2D
* 多新增兩層hidden layer
* loss function改用Cross Entropy取代MSE